In [1]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_awilki13" #please modify so you store data in your collection
coll2name = "sfprj_awilki13"
letter = 'h'

# beginning page index
begin = "1"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]
coll2 = db[coll2name]


gitlab_url = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

gleft = 20

source_url = "https://sourceforge.net/directory/?q=" + letter + "&sort=name&page="
rest_url = "https://sourceforge.net/rest/p/"

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def project_exists(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False


# send queries and extract urls 
def get_gitlab(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    project_count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(letter):
                    if project_exists(el['http_url_to_repo']):
                        project_count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        if project_count >= 50:
                            return
            
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(letter):
                                if project_exists(el['http_url_to_repo']):
                                    project_count += 1
                                    el['forge'] = 'gitlab'
                                    coll.insert_one(el)
                                    if project_count >= 50:
                                        return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')
            
        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
def get_sourcefg(url, coll, rest):
    page = 1
    project_count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_="project-icon", href=True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(letter):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    coll.insert_one(info)
                    project_count += 1
                    if project_count >= 50:
                        return
        page += 1
    return
    
#start retrieving        
get_gitlab(gitlab_url,coll)
get_sourcefg(source_url, coll2, rest_url)
#print collected data
for doc in coll.find({}):
    print(doc)

{'http_url_to_repo': 'https://gitlab.com/MyDesigns/hami.git', 'description': 'A magazine like design project. #oldproject', 'forks_count': 0, 'readme_url': None, 'namespace': {'full_path': 'MyDesigns', 'kind': 'group', 'path': 'MyDesigns', 'id': 3827223, 'name': 'MyDesigns', 'parent_id': None}, 'path_with_namespace': 'MyDesigns/hami', 'default_branch': None, 'tag_list': [], '_id': ObjectId('5bc779f3bd6f6700a86a96ca'), 'created_at': '2018-10-17T17:55:51.676Z', 'name': 'Hami', 'avatar_url': 'https://assets.gitlab-static.net/uploads/-/system/project/avatar/8917334/logo.jpg', 'star_count': 0, 'path': 'hami', 'last_activity_at': '2018-10-17T17:55:51.676Z', 'forge': 'gitlab', 'web_url': 'https://gitlab.com/MyDesigns/hami', 'ssh_url_to_repo': 'git@gitlab.com:MyDesigns/hami.git', 'name_with_namespace': 'MyDesigns / Hami', 'id': 8917334}
{'http_url_to_repo': 'https://gitlab.com/mvsmal/haskero.git', 'description': 'VSCode extension for the Haskell language based on intero\r\n\r\nhttps://marketpl

In [3]:
from pprint import pprint
for doc in coll.find({}):
    pprint(doc)

{'_id': ObjectId('5bc779f3bd6f6700a86a96ca'),
 'avatar_url': 'https://assets.gitlab-static.net/uploads/-/system/project/avatar/8917334/logo.jpg',
 'created_at': '2018-10-17T17:55:51.676Z',
 'default_branch': None,
 'description': 'A magazine like design project. #oldproject',
 'forge': 'gitlab',
 'forks_count': 0,
 'http_url_to_repo': 'https://gitlab.com/MyDesigns/hami.git',
 'id': 8917334,
 'last_activity_at': '2018-10-17T17:55:51.676Z',
 'name': 'Hami',
 'name_with_namespace': 'MyDesigns / Hami',
 'namespace': {'full_path': 'MyDesigns',
               'id': 3827223,
               'kind': 'group',
               'name': 'MyDesigns',
               'parent_id': None,
               'path': 'MyDesigns'},
 'path': 'hami',
 'path_with_namespace': 'MyDesigns/hami',
 'readme_url': None,
 'ssh_url_to_repo': 'git@gitlab.com:MyDesigns/hami.git',
 'star_count': 0,
 'tag_list': [],
 'web_url': 'https://gitlab.com/MyDesigns/hami'}
{'_id': ObjectId('5bc779f4bd6f6700a86a96cb'),
 'avatar_url': 'htt

 'default_branch': 'master',
 'description': 'prueba de Reciclanet',
 'forge': 'gitlab',
 'forks_count': 0,
 'http_url_to_repo': 'https://gitlab.com/reciclanet/hello-world.git',
 'id': 8915944,
 'last_activity_at': '2018-10-17T16:33:39.287Z',
 'name': 'hello-world',
 'name_with_namespace': 'Reciclanet Asociacion Educativa / hello-world',
 'namespace': {'full_path': 'reciclanet',
               'id': 2811512,
               'kind': 'user',
               'name': 'reciclanet',
               'parent_id': None,
               'path': 'reciclanet'},
 'path': 'hello-world',
 'path_with_namespace': 'reciclanet/hello-world',
 'readme_url': 'https://gitlab.com/reciclanet/hello-world/blob/master/README.md',
 'ssh_url_to_repo': 'git@gitlab.com:reciclanet/hello-world.git',
 'star_count': 0,
 'tag_list': [],
 'web_url': 'https://gitlab.com/reciclanet/hello-world'}
{'_id': ObjectId('5bc77a12bd6f6700a86a96d8'),
 'avatar_url': 'https://assets.gitlab-static.net/uploads/-/system/project/avatar/8915728/

               'kind': 'user',
               'name': 'burakb',
               'parent_id': None,
               'path': 'burakb'},
 'path': 'hello-world',
 'path_with_namespace': 'burakb/hello-world',
 'readme_url': 'https://gitlab.com/burakb/hello-world/blob/master/README.md',
 'ssh_url_to_repo': 'git@gitlab.com:burakb/hello-world.git',
 'star_count': 1,
 'tag_list': [],
 'web_url': 'https://gitlab.com/burakb/hello-world'}
{'_id': ObjectId('5bc77a6dbd6f6700a86a96e5'),
 'avatar_url': None,
 'created_at': '2018-10-17T10:08:48.833Z',
 'default_branch': None,
 'description': ' A Pyret-Implementation On Top Of The JVM; Mostly Targeted As '
                'An Interop Scripting Lang For Terasform. ',
 'forge': 'gitlab',
 'forks_count': 0,
 'http_url_to_repo': 'https://gitlab.com/Terasform/hoppr.git',
 'id': 8908809,
 'last_activity_at': '2018-10-17T10:08:48.833Z',
 'name': 'hoppr',
 'name_with_namespace': ' Terasform Education-Engine. / hoppr',
 'namespace': {'full_path': 'Terasform',
    

 'web_url': 'https://gitlab.com/paulmcquad/hiphop-php'}
{'_id': ObjectId('5bc77a84bd6f6700a86a96ee'),
 'avatar_url': None,
 'created_at': '2018-10-17T08:19:57.544Z',
 'default_branch': 'master',
 'description': '',
 'forge': 'gitlab',
 'forks_count': 0,
 'http_url_to_repo': 'https://gitlab.com/AM25/herokuapp-arnita.git',
 'id': 8906565,
 'last_activity_at': '2018-10-17T08:19:57.544Z',
 'name': 'herokuApp-arnita',
 'name_with_namespace': 'Arnita Martiana / herokuApp-arnita',
 'namespace': {'full_path': 'AM25',
               'id': 2565310,
               'kind': 'user',
               'name': 'AM25',
               'parent_id': None,
               'path': 'AM25'},
 'path': 'herokuapp-arnita',
 'path_with_namespace': 'AM25/herokuapp-arnita',
 'readme_url': None,
 'ssh_url_to_repo': 'git@gitlab.com:AM25/herokuapp-arnita.git',
 'star_count': 0,
 'tag_list': [],
 'web_url': 'https://gitlab.com/AM25/herokuapp-arnita'}
{'_id': ObjectId('5bc77a8dbd6f6700a86a96ef'),
 'avatar_url': None,
 'crea

 'last_activity_at': '2018-10-17T03:25:02.462Z',
 'name': 'hugo',
 'name_with_namespace': 'Daniel Scott / hugo',
 'namespace': {'full_path': 'Daniel.Scott',
               'id': 489502,
               'kind': 'user',
               'name': 'Daniel.Scott',
               'parent_id': None,
               'path': 'Daniel.Scott'},
 'path': 'hugo',
 'path_with_namespace': 'Daniel.Scott/hugo',
 'readme_url': 'https://gitlab.com/Daniel.Scott/hugo/blob/master/README.md',
 'ssh_url_to_repo': 'git@gitlab.com:Daniel.Scott/hugo.git',
 'star_count': 0,
 'tag_list': [],
 'web_url': 'https://gitlab.com/Daniel.Scott/hugo'}
{'_id': ObjectId('5bc77ababd6f6700a86a96f8'),
 'avatar_url': None,
 'created_at': '2018-10-17T03:24:55.567Z',
 'default_branch': 'master',
 'description': 'Highcharts JS, the JavaScript charting framework',
 'forge': 'gitlab',
 'forks_count': 0,
 'http_url_to_repo': 'https://gitlab.com/syazwan0913/highcharts.git',
 'id': 8902693,
 'last_activity_at': '2018-10-17T03:24:55.567Z',
 'n

In [4]:
for doc in coll2.find({}):
    pprint(doc)

{'_id': '5165789f34309d5bef93eae0',
 'categories': {'audience': [],
                'database': [],
                'developmentstatus': [],
                'environment': [],
                'language': [],
                'license': [],
                'os': [],
                'topic': [],
                'translation': []},
 'creation_date': '2011-02-28',
 'developers': [{'name': 'Hosam Suliman',
                 'url': 'https://sourceforge.net/u/hossamzee/',
                 'username': 'hossamzee'}],
 'external_homepage': 'https://h-cryptography.sourceforge.io',
 'forge': 'sourceforge',
 'icon_url': None,
 'labels': [],
 'moved_to_url': '',
 'name': 'Cryptography',
 'preferred_support_tool': '',
 'preferred_support_url': '',
 'private': False,
 'screenshots': [{'caption': 'Cipher Shift',
                  'thumbnail_url': 'https://sourceforge.net/p/h-cryptography/screenshot/297135.jpg/thumb',
                  'url': 'https://sourceforge.net/p/h-cryptography/screenshot/297135.jpg

            'tool_label': 'Files',
            'url': '/p/haplinker/files/'},
           {'icons': {'24': 'images/wiki_24.png',
                      '32': 'images/wiki_32.png',
                      '48': 'images/wiki_48.png'},
            'installable': True,
            'mount_label': 'Wiki',
            'mount_point': 'wiki',
            'name': 'wiki',
            'tool_label': 'Wiki',
            'url': '/p/haplinker/wiki/'},
           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Summary',
            'mount_point': 'summary',
            'name': 'summary',
            'sourceforge_group_id': 350965,
            'tool_label': 'Summary',
            'url': '/p/haplinker/summary/'},
           {'icons': {'24': 'images/admin_24.png',
                      '32': 'images/admin_32.png',
  

                                                   'Pre-Alpha',
                                       'id': 8,
                                       'shortname': 'prealpha'}],
                'environment': [],
                'language': [{'fullname': 'C++',
                              'fullpath': 'Programming Language :: C++',
                              'id': 165,
                              'shortname': 'cpp'}],
                'license': [{'fullname': 'GNU Library or Lesser General Public '
                                         'License version 2.0 (LGPLv2)',
                             'fullpath': 'License :: OSI-Approved Open Source '
                                         ':: GNU Library or Lesser General '
                                         'Public License version 2.0 (LGPLv2)',
                             'id': 16,
                             'shortname': 'lgpl'}],
                'os': [{'fullname': 'All POSIX (Linux/BSD/UNIX-like OSes)',
              

                      '32': 'images/wiki_32.png',
                      '48': 'images/wiki_48.png'},
            'installable': True,
            'mount_label': 'Wiki',
            'mount_point': 'wiki',
            'name': 'wiki',
            'tool_label': 'Wiki',
            'url': '/p/hsharp/wiki/'},
           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Support',
            'mount_point': 'support',
            'name': 'support',
            'tool_label': 'Support',
            'url': '/p/hsharp/support/'},
           {'icons': {'24': 'images/downloads_24.png',
                      '32': 'images/downloads_32.png',
                      '48': 'images/downloads_48.png'},
            'installable': False,
            'mount_label': 'Files',
            'mount_point': 'files',
          

 'status': 'active',
 'summary': '',
 'tools': [{'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Reviews',
            'mount_point': 'reviews',
            'name': 'reviews',
            'tool_label': 'Reviews',
            'url': '/p/h-a-del/reviews/'},
           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Summary',
            'mount_point': 'summary',
            'name': 'summary',
            'sourceforge_group_id': 2589273,
            'tool_label': 'Summary',
            'url': '/p/h-a-del/summary/'},
           {'icons': {'24': 'images/code_24.png',
                      '32': 'images/code

...

           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Support',
            'mount_point': 'support',
            'name': 'support',
            'tool_label': 'Support',
            'url': '/p/hcs12hallibrary/support/'},
           {'icons': {'24': 'images/admin_24.png',
                      '32': 'images/admin_32.png',
                      '48': 'images/admin_48.png'},
            'installable': False,
            'mount_label': 'Activity',
            'mount_point': 'activity',
            'name': 'activity',
            'tool_label': 'Tool',
            'url': '/p/hcs12hallibrary/activity/'}],
 'url': 'https://sourceforge.net/p/hcs12hallibrary/',
 'video_url': ''}
{'_id': '51269326271846353893c9dc',
 'categories': {'audience': [{'fullname': 'Developers',
                             

            'url': '/p/hirbtextracter/summary/'},
           {'icons': {'24': 'images/tickets_24.png',
                      '32': 'images/tickets_32.png',
                      '48': 'images/tickets_48.png'},
            'installable': True,
            'mount_label': 'Tickets',
            'mount_point': 'tickets',
            'name': 'tickets',
            'tool_label': 'Tickets',
            'url': '/p/hirbtextracter/tickets/'},
           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Support',
            'mount_point': 'support',
            'name': 'support',
            'tool_label': 'Support',
            'url': '/p/hirbtextracter/support/'},
           {'icons': {'24': 'images/code_24.png',
                      '32': 'images/code_32.png',
                      '48': 'images/code_4

            'mount_point': 'code',
            'name': 'svn',
            'tool_label': 'SVN',
            'url': '/p/h-j-p/code/'}],
 'url': 'https://sourceforge.net/p/h-j-p/',
 'video_url': None}
{'_id': '52c54122b9363c4dc6fba1eb',
 'categories': {'audience': [],
                'database': [],
                'developmentstatus': [],
                'environment': [],
                'language': [],
                'license': [],
                'os': [],
                'topic': [],
                'translation': []},
 'creation_date': '2014-01-02',
 'developers': [],
 'external_homepage': None,
 'forge': 'sourceforge',
 'icon_url': None,
 'labels': [],
 'moved_to_url': '',
 'name': 'HMC',
 'preferred_support_tool': 'wiki',
 'preferred_support_url': '',
 'private': False,
 'screenshots': [],
 'short_description': '',
 'shortname': 'h-m-c',
 'socialnetworks': [{'accounturl': '', 'socialnetwork': 'Twitter'},
                    {'accounturl': None, 'socialnetwork': 'Facebook'}],
 'st

            'tool_label': 'Summary',
            'url': '/p/hpkgsrc/summary/'},
           {'icons': {'24': 'images/admin_24.png',
                      '32': 'images/admin_32.png',
                      '48': 'images/admin_48.png'},
            'installable': False,
            'mount_label': 'Activity',
            'mount_point': 'activity',
            'name': 'activity',
            'tool_label': 'Tool',
            'url': '/p/hpkgsrc/activity/'},
           {'icons': {'24': 'images/wiki_24.png',
                      '32': 'images/wiki_32.png',
                      '48': 'images/wiki_48.png'},
            'installable': True,
            'mount_label': 'Wiki',
            'mount_point': 'wiki',
            'name': 'wiki',
            'tool_label': 'Wiki',
            'url': '/p/hpkgsrc/wiki/'},
           {'icons': {'24': 'images/code_24.png',
                      '32': 'images/code_32.png',
                      '48': 'images/code_48.png'},
            'installable': True,
    

                      '48': 'images/downloads_48.png'},
            'installable': False,
            'mount_label': 'Files',
            'mount_point': 'files',
            'name': 'files',
            'tool_label': 'Files',
            'url': '/p/h-child-abuse/files/'},
           {'icons': {'24': 'images/wiki_24.png',
                      '32': 'images/wiki_32.png',
                      '48': 'images/wiki_48.png'},
            'installable': True,
            'mount_label': 'Wiki',
            'mount_point': 'wiki',
            'name': 'wiki',
            'tool_label': 'Wiki',
            'url': '/p/h-child-abuse/wiki/'},
           {'icons': {'24': 'images/sftheme/24x24/blog_24.png',
                      '32': 'images/sftheme/32x32/blog_32.png',
                      '48': 'images/sftheme/48x48/blog_48.png'},
            'installable': False,
            'mount_label': 'Summary',
            'mount_point': 'summary',
            'name': 'summary',
            'sourceforge_group_